Notebook based on _Hands-On Graph Neural Networks Using Python_, by Maxime Labonne.

# Ch 4. Improving Embeddings with Biased Random Walks in Node2Vec

In [1]:
from collections import defaultdict
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import matplotlib.pyplot as plt
import networkx as nx
from node2vec import Node2Vec
import pandas as pd

/home/willie/.cache/pypoetry/virtualenvs/gnn-N42gmvSp-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 4.5 Application: Movie recommendation engine

### 4.5.1 Download data file

### 4.5.2 Read movies dataframe

In [2]:
movies = pd.read_csv(
    'ml-100k/u.item', sep='|', usecols=range(2),
    names=['movie_id', 'title'], encoding='latin-1')
movies

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


### 4.5.3 Read ratings dataframe

In [3]:
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t',
    names=['user_id', 'movie_id', 'rating', 'unix_timestamp'])
ratings

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [4]:
ratings = ratings[ratings.rating >= 4]
ratings

,user_id,movie_id,rating,unix_timestamp
5,298,474,4,884182806
7,253,465,5,891628467
11,286,1014,5,879781125
12,200,222,5,876042340
16,122,387,5,879270459
...,...,...,...,...
99988,421,498,4,892241344
99989,495,1091,4,888637503
99990,806,421,4,882388897
99991,676,538,4,892685437


### 4.5.4 Count coincident movie pairs

That is, # of times a single person liked both movies in the pair.

In [5]:
pairs = defaultdict(int)

for group in ratings.groupby('user_id'):
    
    # group = rows representing the movies that the current user likes
    user_movies = list(group[1]['movie_id'])

    # The book neglects this step, but we need it to ensure that (a, b) and (b, a)
    # are treated as equivalents.
    
    user_movies.sort()
    # print(f'Person {group[0]}: {user_movies}')
    # print(group[1])
    # print('-----')

    # Increment each (movie_i, movie_j) pair
    for i in range(len(user_movies)):
        for j in range(i+1, len(user_movies)):
            pairs[(user_movies[i], user_movies[j])] += 1

# Convert to normal dict to avoid side effect where queries add new dict rows
pairs = dict(pairs)

In [6]:
list(pairs)[0:5]

[(1, 3), (1, 6), (1, 7), (1, 9), (1, 12)]

In [7]:
pairs[(1, 3)]

19

### 4.5.5 Build weighted graph

In [8]:
G = nx.Graph()
for pair, score in pairs.items():
    if score >= 50:
        G.add_edge(pair[0], pair[1], weight=score)

In [9]:
print(G)

Graph with 269 nodes and 7164 edges


### 4.5.6 Calculate similarity using node2vec

In [10]:
# node2vec automatically generates random walks based on walk_length, num_walks, p and q.
# It uses the random walks to generate node embeddings driven by neighborhood topology.
node2vec = Node2Vec(G, dimensions=64, walk_length=20, num_walks=200, p=2, q=1, workers=1)

Generating walks (CPU: 1): 100%|█████████████████████████████████████████████████████████████████████████████| 200/200 [00:15<00:00, 12.83it/s]


In [11]:
# window=10: 5 nodes before, 5 nodes after
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [12]:
# movies[movies.title == 'Star Wars (1977)'].movie_id.values[0]

In [13]:
G.nodes

NodeView((1, 7, 9, 12, 14, 15, 22, 23, 25, 28, 50, 56, 58, 64, 79, 82, 87, 88, 89, 91, 95, 96, 98, 100, 111, 121, 124, 127, 132, 133, 134, 135, 137, 144, 150, 151, 156, 161, 168, 169, 172, 173, 174, 175, 176, 178, 181, 182, 183, 185, 186, 187, 190, 191, 193, 194, 195, 196, 197, 199, 202, 203, 204, 208, 209, 210, 216, 222, 223, 228, 230, 234, 238, 248, 250, 257, 258, 265, 268, 269, 42, 55, 93, 129, 154, 246, 86, 47, 48, 157, 192, 198, 13, 66, 68, 177, 241, 239, 65, 81, 109, 170, 184, 227, 77, 237, 273, 275, 276, 282, 283, 286, 293, 300, 302, 313, 285, 255, 272, 284, 301, 242, 316, 303, 310, 318, 328, 11, 288, 357, 271, 294, 24, 70, 153, 211, 385, 408, 423, 432, 433, 435, 443, 428, 430, 455, 233, 8, 71, 180, 274, 419, 427, 474, 475, 479, 480, 483, 496, 498, 511, 515, 523, 527, 514, 484, 136, 213, 410, 462, 478, 482, 485, 501, 509, 510, 520, 521, 528, 531, 519, 517, 4, 69, 97, 179, 568, 603, 655, 31, 125, 164, 188, 200, 215, 403, 471, 588, 651, 657, 367, 92, 317, 566, 654, 99, 393, 402, 4

In [14]:
# The movie IDs in the graph are integers
G.nodes[479]

{}

In [15]:
# The movie IDs in the dataset are integers
movies[movies.movie_id == 7]

,movie_id,title
6,7,Twelve Monkeys (1995)


In [16]:
# IMPORTANT: The movie IDs in the Word2Vec model are strings, since these are "word" vectors.
model.wv['50']

array([ 2.2476378e-01, -8.9162201e-02,  4.1044569e-01,  3.0132774e-01,
       -1.5145986e-01, -3.5008621e-01,  2.4964688e-02,  2.9324040e-01,
       -2.4738403e-01,  2.5566768e-02,  1.9483520e-01, -1.7182132e-02,
        9.0133578e-02,  2.5519046e-01, -3.7705373e-02,  2.4454416e-01,
        2.0123713e-01,  6.4065814e-02, -1.2974530e-01, -6.0934804e-02,
       -4.4112213e-02,  2.2956261e-01,  6.2019195e-02, -3.6585692e-01,
        1.5911461e-01, -5.9932568e-03, -1.3207661e-01,  1.3796027e-01,
        9.6409857e-02, -3.1683946e-01,  2.3814124e-01,  6.5281063e-02,
        2.8441069e-01, -1.2331616e-01, -1.9545284e-01, -1.7240395e-01,
       -6.8942979e-03, -9.5372699e-02,  9.4249569e-02, -1.0248673e-01,
       -1.3871904e-03, -8.8270791e-02, -2.2382425e-02,  5.7902308e-03,
        1.3782537e-01,  1.6370797e-01, -2.8012010e-01,  4.1039059e-01,
       -1.3471824e-01, -1.1763230e-02, -1.6609277e-01,  1.5283011e-01,
       -1.0150678e-03, -2.4592620e-01,  3.8073529e-02, -1.4543892e-01,
      

In [17]:
model.wv.most_similar('50')

[('174', 0.5930283069610596),
 ('100', 0.5560889840126038),
 ('181', 0.5319597721099854),
 ('56', 0.43334922194480896),
 ('172', 0.42717882990837097),
 ('98', 0.4141160249710083),
 ('1012', 0.4041602313518524),
 ('746', 0.392008513212204),
 ('845', 0.38912248611450195),
 ('237', 0.38140907883644104)]

In [18]:
def recommend(model, movies, movie):
    # IMPORTANT: The movie IDs need to be strings!
    # Otherwise the recommendations will be wrong.
    movie_id = str(movies[movies.title == movie].movie_id.values[0])
    print(f'Recommendations for [{movie_id:>4}] {movie}:')
    for id_score in model.wv.most_similar(movie_id):  #[:5]:
        id, score = id_score
        title = movies[movies.movie_id == int(id)].title.values[0]
        print(f'- [{id:>4}] {title}: {score:.3f}')

In [19]:
recommend(model, movies, 'Star Wars (1977)')

Recommendations for [  50] Star Wars (1977):
- [ 174] Raiders of the Lost Ark (1981): 0.593
- [ 100] Fargo (1996): 0.556
- [ 181] Return of the Jedi (1983): 0.532
- [  56] Pulp Fiction (1994): 0.433
- [ 172] Empire Strikes Back, The (1980): 0.427
- [  98] Silence of the Lambs, The (1991): 0.414
- [1012] Private Parts (1997): 0.404
- [ 746] Real Genius (1985): 0.392
- [ 845] That Thing You Do! (1996): 0.389
- [ 237] Jerry Maguire (1996): 0.381


In [20]:
recommend(model, movies, 'Raiders of the Lost Ark (1981)')

Recommendations for [ 174] Raiders of the Lost Ark (1981):
- [  50] Star Wars (1977): 0.593
- [ 172] Empire Strikes Back, The (1980): 0.582
- [  56] Pulp Fiction (1994): 0.579
- [  98] Silence of the Lambs, The (1991): 0.518
- [ 173] Princess Bride, The (1987): 0.490
- [  79] Fugitive, The (1993): 0.488
- [  96] Terminator 2: Judgment Day (1991): 0.474
- [ 181] Return of the Jedi (1983): 0.473
- [  69] Forrest Gump (1994): 0.470
- [ 423] E.T. the Extra-Terrestrial (1982): 0.454


In [21]:
recommend(model, movies, 'Toy Story (1995)')

Recommendations for [   1] Toy Story (1995):
- [ 121] Independence Day (ID4) (1996): 0.594
- [  28] Apollo 13 (1995): 0.588
- [ 181] Return of the Jedi (1983): 0.584
- [ 423] E.T. the Extra-Terrestrial (1982): 0.579
- [  56] Pulp Fiction (1994): 0.546
- [   7] Twelve Monkeys (1995): 0.526
- [ 173] Princess Bride, The (1987): 0.502
- [ 168] Monty Python and the Holy Grail (1974): 0.500
- [  12] Usual Suspects, The (1995): 0.499
- [ 237] Jerry Maguire (1996): 0.495


In [22]:
recommend(model, movies, 'Return of the Jedi (1983)')

Recommendations for [ 181] Return of the Jedi (1983):
- [ 172] Empire Strikes Back, The (1980): 0.592
- [  79] Fugitive, The (1993): 0.591
- [   1] Toy Story (1995): 0.584
- [ 173] Princess Bride, The (1987): 0.580
- [  96] Terminator 2: Judgment Day (1991): 0.573
- [ 210] Indiana Jones and the Last Crusade (1989): 0.564
- [  50] Star Wars (1977): 0.532
- [ 168] Monty Python and the Holy Grail (1974): 0.518
- [ 237] Jerry Maguire (1996): 0.514
- [ 204] Back to the Future (1985): 0.510


In [23]:
recommend(model, movies, '2001: A Space Odyssey (1968)')

Recommendations for [ 135] 2001: A Space Odyssey (1968):
- [ 179] Clockwork Orange, A (1971): 0.711
- [ 185] Psycho (1960): 0.699
- [ 435] Butch Cassidy and the Sundance Kid (1969): 0.659
- [ 180] Apocalypse Now (1979): 0.623
- [ 427] To Kill a Mockingbird (1962): 0.604
- [  23] Taxi Driver (1976): 0.594
- [ 357] One Flew Over the Cuckoo's Nest (1975): 0.592
- [ 194] Sting, The (1973): 0.577
- [  89] Blade Runner (1982): 0.571
- [ 187] Godfather: Part II, The (1974): 0.567


## 4.6 Further reading

- [node2vec: Scalable Feature Learning for Networks](https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf)
- [The MovieLens Datasets: History and Context](https://dl.acm.org/doi/10.1145/2827872)